In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

In [4]:
from transformers import pipeline

context="""
In the year 1985, Deva and Vardharaja Mannar are inseperable companions in Khansaar, a puissant city-state ruled by Vardha's father Raja Mannar. After Raja Mannar commands the decimation of Shouryaanga tribe, a mob pounces on Deva and his mother at their residence, but Vardha intercedes for them and confers his important territory in exchange for sparing their lives. Before exiting Khansaar, Deva vows to return on Vardha's call.

In 2017, a business tycoon Krishnakanth learns that his daughter Aadhya arrived at Varanasi from New York to finish her mother's last rites. This alerts Krishnakanth's old rivals, who conspire to abduct her, but Krishnakanth enlists the assistance of Bilal, who thwarts the enemies' attempts to seize her and safeguards her at his friend Deva's place in Tinsukia, Assam. Aadhya pretends to be an English teacher at a school, where Deva's mother is the headmaster. Eventually, Aadhya is traced to Tinsukia where upon his mother's reluctant orders, Deva hinders the goons' attempts to capture her. Dreading for Deva, Deva's mother plans to abscond with him, while Aadhya and Bilal are carried off in a convoy led by a man named Rinda and one that is sealed with the mysterious emblem of Khansaar.

Witnessing this, Deva's mother signals him to rescue Aadhya and Bilal. Deva refrains the convoy, grabbing the attention of Vardha and his step-sister Radha Rama Mannar, who has ordered Aadhya's abduction to avenge something Krishnakanth has committed against her in Deva's favour seven years ago. Radha admits to strategizing the events to pit Deva and Vardha against each other since opposing the seal of Khansaar is a crime and the perpetrator has to be killed, a rule that Deva himself is told to have imposed years ago. After the rescue, Bilal recounts the tale of Khansaar and Deva and Vardha's past for Aadhya.
"""

quan_pipeline = pipeline("question-answering",
                         model="deepset/minilm-uncased-squad2")

answer=quan_pipeline(question="whose daughter is Aadhya ?",
             context=context)
print(answer)


{'score': 0.9054983258247375, 'start': 462, 'end': 474, 'answer': 'Krishnakanth'}


In [5]:
print("\nAnother question :")
print(quan_pipeline( question="who is vardhas father",
             context=context))


Another question :
{'score': 0.6489396095275879, 'start': 133, 'end': 144, 'answer': 'Raja Mannar'}


In [10]:
print("\nAnother question :")
print(quan_pipeline( question="where do  vardha and deva live?",
             context=context))


Another question :
{'score': 0.5676174759864807, 'start': 76, 'end': 84, 'answer': 'Khansaar'}


In [7]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [8]:
from evaluate import load
squad_metric = load("squad_v2")

#Ignoring Context & Question as they are not needed for evaluation
#This example is to showcase how the evaluation works based on match between the prediction
#and the correct answer

correct_answer="Paris"

predicted_answers=["Paris",
                 "London",
                 "Paris is one of the best cities in the world"]

cum_predictions=[]
cum_references=[]
for i in range(len(predicted_answers)):

    #Use the input format for predictions
    predictions = [{'prediction_text':predicted_answers[i],
                    'id': str(i),
                    'no_answer_probability': 0.}]
    cum_predictions.append(predictions[0])

    #Use the input format for naswers
    references = [{'answers': {'answer_start': [1],
                               'text': [correct_answer]},
                   'id': str(i)}]
    cum_references.append(references[0])

    results = squad_metric.compute(predictions=predictions,
                                   references=references)
    print("F1 is", results.get('f1'),
          " for answer :", predicted_answers[i])

#Compute for cumulative Results
cum_results=squad_metric.compute(predictions=cum_predictions,
                                 references=cum_references)
print("\n Cumulative Results : \n",cum_results)

F1 is 100.0  for answer : Paris
F1 is 0.0  for answer : London
F1 is 22.22222222222222  for answer : Paris is one of the best cities in the world

 Cumulative Results : 
 {'exact': 33.333333333333336, 'f1': 40.74074074074074, 'total': 3, 'HasAns_exact': 33.333333333333336, 'HasAns_f1': 40.74074074074074, 'HasAns_total': 3, 'best_exact': 33.333333333333336, 'best_exact_thresh': 0.0, 'best_f1': 40.74074074074074, 'best_f1_thresh': 0.0}
